In [ ]:
Cài đặt

In [ ]:
!pip install -q datasets pillow matplotlib tensorflow

Import thư viện cần thiết

In [12]:
# Set cache directory to D drive
import os
from pathlib import Path
CACHE_DIR = Path("D:/huggingface_cache")
CACHE_DIR.mkdir(exist_ok=True)
os.environ['HF_HOME'] = str(CACHE_DIR)
os.environ['HF_DATASETS_CACHE'] = str(CACHE_DIR / 'datasets')

from datasets import load_dataset
import numpy as np
import tensorflow as tf
from tensorflow import keras
import PIL
from PIL import ImageFile
import matplotlib.pyplot as plt

# Cho phép load ảnh bị truncated/incomplete
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Setup model save path
BASE_DIR = Path.cwd()  # Thư mục hiện tại của notebook
MODEL_SAVE_DIR = BASE_DIR / 'models'
MODEL_SAVE_DIR.mkdir(exist_ok=True)
print(f"Models will be saved to: {MODEL_SAVE_DIR}")

Models will be saved to: d:\My WorkSpace\Littera\models


Load dataset

In [13]:
# Load 1 triệu mẫu từ dataset (cache sẽ dùng từ D:/huggingface_cache)
print("Loading 1,000,000 samples from MJSynth dataset...")
ds = load_dataset("priyank-m/MJSynth_text_recognition", split="train[:1000000]")
print(f"Loaded {len(ds)} samples successfully!")

Loading 1,000,000 samples from MJSynth dataset...
Loaded 1000000 samples successfully!
Loaded 1000000 samples successfully!


In [ ]:
# Kiểm tra phân bố chữ hoa/thường trong dataset
import random
sample_labels = [ds[i]['label'] for i in random.sample(range(len(ds)), 100)]

uppercase_count = sum(1 for label in sample_labels if any(c.isupper() for c in label))
lowercase_count = sum(1 for label in sample_labels if any(c.islower() for c in label))
all_upper = sum(1 for label in sample_labels if label.isupper() and label.isalpha())
all_lower = sum(1 for label in sample_labels if label.islower() and label.isalpha())
mixed_case = sum(1 for label in sample_labels if any(c.isupper() for c in label) and any(c.islower() for c in label))

print(f"Sample của 100 labels:")
print(f"  Có chữ HOA: {uppercase_count}/100")
print(f"  Có chữ thường: {lowercase_count}/100")
print(f"  TOÀN HOA: {all_upper}/100")
print(f"  toàn thường: {all_lower}/100")
print(f"  Trộn lẫn: {mixed_case}/100")
print(f"\nVí dụ 10 labels đầu:")
for label in sample_labels[:10]:
    print(f"  '{label}'")

Mã hoá

In [15]:
# CHARSET phải bao gồm CHỮ HOA + chữ thường + số
charset = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789"
char_to_id = {c: i for i, c in enumerate(charset)}  # nhãn 0..len(charset)-1, blank sẽ là lớp cuối cùng của softmax
num_classes = len(charset)  # model Dense sẽ dùng num_classes + 1 (thêm blank)

def encode_text(t: str):
    # KHÔNG .lower() nữa - giữ nguyên chữ hoa/thường
    return [char_to_id[c] for c in t if c in char_to_id]

In [ ]:
# (TÙY CHỌN) Data augmentation: cân bằng chữ hoa/thường
# Chỉ dùng nếu model sau 20 epochs vẫn output toàn HOA
import random

def augment_case(text):
    """Thêm biến thể case để cân bằng dataset"""
    if random.random() < 0.3:  # 30% giữ nguyên
        return text
    elif random.random() < 0.5:  # 20% chuyển toàn chữ thường
        return text.lower()
    elif random.random() < 0.7:  # 20% chuyển toàn chữ HOA
        return text.upper()
    else:  # 30% random case từng ký tự
        return ''.join(c.upper() if random.random() > 0.5 else c.lower() for c in text)

# Uncomment dòng dưới trong hàm encode_text nếu cần:
# t = augment_case(t)  # Thêm vào trước return

Tham số ảnh và pipeline


In [16]:
IMG_HEIGHT, IMG_WIDTH = 32, 128
BATCH_SIZE = 64

def gen(examples):
    for ex in examples:
        try:
            img: PIL.Image.Image = ex["image"].convert("L").resize((IMG_WIDTH, IMG_HEIGHT), PIL.Image.BILINEAR)
            img = np.array(img, dtype=np.float32) / 255.0
            img = img[..., None]  # (H, W, 1)
            label_ids = np.array(encode_text(ex["label"]), dtype=np.int32)
            
            # Bỏ qua nếu label rỗng
            if len(label_ids) == 0:
                continue
                
            yield img, label_ids, np.int32(len(label_ids))
        except (OSError, PIL.UnidentifiedImageError, Exception) as e:
            # Bỏ qua ảnh bị lỗi/hỏng
            continue

Trai val/split


In [17]:
# Split 90% train / 10% validation
print("Splitting dataset into train/validation...")
split = ds.train_test_split(test_size=0.1, seed=42)
train_raw, val_raw = split["train"], split["test"]
print(f"Train samples: {len(train_raw)}, Validation samples: {len(val_raw)}")

def make_tfds(hf_dataset):
    output_signature = (
        tf.TensorSpec(shape=(IMG_HEIGHT, IMG_WIDTH, 1), dtype=tf.float32),    # image
        tf.TensorSpec(shape=(None,), dtype=tf.int32),                          # label (variable length)
        tf.TensorSpec(shape=(), dtype=tf.int32),                               # label_length
    )
    ds_tf = tf.data.Dataset.from_generator(lambda: gen(hf_dataset), output_signature=output_signature)
    ds_tf = ds_tf.padded_batch(
        BATCH_SIZE,
        padded_shapes=(
            (IMG_HEIGHT, IMG_WIDTH, 1),   # image
            (None,),                      # label padded
            (),                           # label_length
        ),
        padding_values=(
            0.0,     # image pad
            -1,      # label pad value expected by ctc_label_dense_to_sparse
            0,       # label_length
        ),
        drop_remainder=True
    ).prefetch(tf.data.AUTOTUNE)
    return ds_tf

print("Creating TensorFlow datasets...")
train_ds = make_tfds(train_raw)
val_ds = make_tfds(val_raw)
print("Datasets ready!")

Splitting dataset into train/validation...
Train samples: 900000, Validation samples: 100000
Creating TensorFlow datasets...
Datasets ready!


Model CRNN

In [18]:
def build_crnn(num_classes):
    inp = keras.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 1), name="image")

    x = keras.layers.Conv2D(64, 3, padding="same", activation="relu")(inp)
    x = keras.layers.MaxPooling2D((2, 2))(x)               # 16x64
    x = keras.layers.Conv2D(128, 3, padding="same", activation="relu")(x)
    x = keras.layers.MaxPooling2D((2, 2))(x)               # 8x32
    x = keras.layers.Conv2D(256, 3, padding="same", activation="relu")(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Conv2D(256, 3, padding="same", activation="relu")(x)
    x = keras.layers.MaxPooling2D((2, 1))(x)               # 4x32
    x = keras.layers.Conv2D(512, 3, padding="same", activation="relu")(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.MaxPooling2D((2, 1))(x)               # 2x32
    x = keras.layers.Conv2D(512, 2, padding="valid", activation="relu")(x)  # 1x31

    # Reshape -> time steps x features (timesteps = 31)
    x = keras.layers.Reshape(target_shape=(31, 512))(x)

    x = keras.layers.Bidirectional(keras.layers.LSTM(256, return_sequences=True))(x)
    x = keras.layers.Bidirectional(keras.layers.LSTM(256, return_sequences=True))(x)

    # +1 cho blank
    out = keras.layers.Dense(num_classes + 1, activation="softmax")(x)  # (B, T, C+1)
    return keras.Model(inp, out, name="crnn")

base_model = build_crnn(num_classes)

Train model với CTC loss

In [19]:
labels = keras.Input(shape=(None,), dtype=tf.int32, name="labels")
input_length = keras.Input(shape=(1,), dtype=tf.int32, name="input_length")
label_length = keras.Input(shape=(1,), dtype=tf.int32, name="label_length")

logits = base_model.output  # (B, T, C+1)
def ctc_loss_layer(args):
    y_true, y_pred, in_len, lab_len = args
    return keras.backend.ctc_batch_cost(y_true, y_pred, in_len, lab_len)

loss_out = keras.layers.Lambda(ctc_loss_layer, name="ctc_loss")([labels, logits, input_length, label_length])

train_model = keras.Model(
    inputs=[base_model.input, labels, input_length, label_length],
    outputs=loss_out,
)

train_model.compile(optimizer=keras.optimizers.Adam(1e-3), loss=lambda y_true, y_pred: y_pred)

Pack batch: thêm input_length (timesteps = 31) và dummy y

In [20]:
TIMESTEPS = 31

def pack_batch(images, labels_batch, label_lens):
    bsz = tf.shape(images)[0]
    in_len = tf.fill([bsz, 1], TIMESTEPS)
    lab_len = tf.expand_dims(label_lens, axis=1)
    inputs = {
        "image": images,
        "labels": labels_batch,
        "input_length": in_len,
        "label_length": lab_len,
    }
    # y dummy (Keras cần target, nhưng loss đã ở outputs)
    y = tf.zeros((bsz, 1), dtype=tf.float32)
    return inputs, y

train_data = train_ds.map(pack_batch)
val_data = val_ds.map(pack_batch)

 Train

# Setup callbacks


In [23]:
checkpoint_path = MODEL_SAVE_DIR / "crnn_ocr_ctc_1m_checkpoint.h5"
final_model_path = MODEL_SAVE_DIR / "crnn_ocr_ctc_1m.h5"

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath=str(checkpoint_path),
        save_best_only=True,
        monitor='val_loss',
        mode='min',
        verbose=1
    ),
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-6,
        verbose=1
    )
]

print("Starting training on 1,000,000 samples...")
print(f"Checkpoint will be saved to: {checkpoint_path}")

history = train_model.fit(
    train_data,
    validation_data=val_data,
    epochs=20,  # Tăng epochs cho dataset lớn
    callbacks=callbacks,
    verbose=1
)
 
# Lưu model cuối cùng
print(f"\nSaving final model to: {final_model_path}")
base_model.save(str(final_model_path))
print("Training complete!")

Starting training on 1,000,000 samples...
Checkpoint will be saved to: d:\My WorkSpace\Littera\models\crnn_ocr_ctc_1m_checkpoint.h5
Epoch 1/20


    110/Unknown 61s 557ms/step - loss: 1.5860

KeyboardInterrupt: 

Resume Training từ Checkpoint (Nếu cần)

In [ ]:
# CHỈ DÙNG NẾU TRAINING BỊ DỪNG/CRASH
# Cell này load checkpoint và tiếp tục train

checkpoint_path = MODEL_SAVE_DIR / "crnn_ocr_ctc_1m_checkpoint.h5"

print(f"Loading checkpoint from: {checkpoint_path}")

# Định nghĩa ctc_loss_layer
def ctc_loss_layer(args):
    y_true, y_pred, in_len, lab_len = args
    return keras.backend.ctc_batch_cost(y_true, y_pred, in_len, lab_len)

# Load checkpoint
loaded_model = keras.models.load_model(
    str(checkpoint_path),
    custom_objects={'ctc_loss_layer': ctc_loss_layer},
    compile=False
)
print("✓ Loaded model from checkpoint")

# Kiểm tra xem là base_model hay train_model
if len(loaded_model.inputs) == 1:
    # Chỉ có 1 input 'image' → đây là base_model
    base_model = loaded_model
    print("✓ Checkpoint is base_model")
else:
    # Nhiều inputs → đây là train_model, cần extract base_model
    print("✓ Extracting base_model from train_model...")
    
    # Tìm input 'image'
    image_input = None
    for inp in loaded_model.inputs:
        if 'image' in inp.name:
            image_input = inp
            break
    
    # Tìm Dense layer cuối cùng (output layer của base model)
    base_output = None
    for layer in reversed(loaded_model.layers):  # Duyệt ngược từ cuối
        if isinstance(layer, keras.layers.Dense):
            # Kiểm tra output shape
            try:
                shape = layer.output.shape  # Dùng layer.output.shape, KHÔNG phải layer.output_shape
                if shape[-1] == (num_classes + 1):  # 63 = 62 chars + 1 blank
                    base_output = layer.output
                    print(f"   Found Dense layer: {layer.name} with output shape {shape}")
                    break
            except:
                continue
    
    if base_output is None:
        raise ValueError("Could not find Dense output layer in checkpoint!")
    
    base_model = keras.Model(inputs=image_input, outputs=base_output)
    print("✓ Extracted base_model successfully")

# Rebuild train_model
labels = keras.Input(shape=(None,), dtype=tf.int32, name="labels")
input_length = keras.Input(shape=(1,), dtype=tf.int32, name="input_length")
label_length = keras.Input(shape=(1,), dtype=tf.int32, name="label_length")

logits = base_model.output

# Lambda layer với output_shape
loss_out = keras.layers.Lambda(
    ctc_loss_layer, 
    output_shape=(1,),
    name="ctc_loss"
)([labels, logits, input_length, label_length])

train_model = keras.Model(
    inputs=[base_model.input, labels, input_length, label_length],
    outputs=loss_out,
)

# Compile với LR thấp hơn
train_model.compile(
    optimizer=keras.optimizers.Adam(5e-4),
    loss=lambda y_true, y_pred: y_pred
)

print("\n✓ Model ready to resume training!")
print(f"   Base model output shape: {base_model.output.shape}")
print(f"   Will train remaining epochs")

Loading checkpoint from: d:\My WorkSpace\Littera\models\crnn_ocr_ctc_1m_checkpoint.h5
✓ Loaded model from checkpoint
Error loading checkpoint: 'Dense' object has no attribute 'output_shape'
✓ Loaded model from checkpoint
Error loading checkpoint: 'Dense' object has no attribute 'output_shape'


AttributeError: 'Dense' object has no attribute 'output_shape'

In [ ]:
# Resume training với callbacks
final_model_path = MODEL_SAVE_DIR / "crnn_ocr_ctc_1m.h5"

callbacks_resume = [
    keras.callbacks.ModelCheckpoint(
        filepath=str(checkpoint_path),
        save_best_only=True,
        monitor='val_loss',
        mode='min',
        verbose=1
    ),
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-6,
        verbose=1
    )
]

print("Resuming training...")
print(f"Training will continue for remaining epochs (7-20)")

# Chạy tiếp 14 epochs (20 - 6)
history_resume = train_model.fit(
    train_data,
    validation_data=val_data,
    epochs=14,  # 20 - 6 = 14 epochs còn lại
    callbacks=callbacks_resume,
    verbose=1
)

# Lưu model cuối cùng
print(f"\nSaving final model to: {final_model_path}")
base_model.save(str(final_model_path))
print("Training complete!")

In [22]:
# Plot training & validation loss
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Model Training History')
plt.legend()
plt.grid(True)
plt.savefig(str(MODEL_SAVE_DIR / 'training_history.png'))
plt.show()

print(f"\nBest validation loss: {min(history.history['val_loss']):.4f}")
print(f"Final model saved at: {final_model_path}")
print(f"Model size: {final_model_path.stat().st_size / (1024**2):.2f} MB")

NameError: name 'history' is not defined

<Figure size 1000x500 with 0 Axes>

Xem vài mẫu

In [ ]:
batch = next(iter(val_ds))
imgs, lab_ids, lab_lens = batch
preds = base_model.predict(imgs)

In [ ]:
# Greedy decode
input_len_np = np.full((preds.shape[0],), TIMESTEPS)
results = keras.backend.ctc_decode(preds, input_length=input_len_np, greedy=True)[0][0].numpy()

# Tạo lại id_to_char từ charset đã cập nhật (có chữ HOA)
id_to_char = {i: c for i, c in enumerate(charset)}

def decode_ids(ids):
    return "".join(id_to_char[i] for i in ids if i != -1)

for i in range(10):
    plt.imshow(imgs[i,...,0], cmap="gray")
    plt.axis("off")
    plt.title(f"pred: {decode_ids(results[i])}")
    plt.show()